In [1]:
import numpy as np

# Reproduzierbarkeit
rng = np.random.default_rng(42)

# -----------------------------
# 1) Datengenerierung
# -----------------------------
# Eingaben: a, b, c ~ N(0,1)
N_train, N_test = 2000, 1000
Xtr = rng.normal(size=(N_train, 3))
Xte = rng.normal(size=(N_test, 3))

In [2]:
Xtr.shape

(2000, 3)

In [3]:
Xtr[0]

array([ 0.30471708, -1.03998411,  0.7504512 ])

In [4]:
Xte.shape

(1000, 3)

In [5]:
a_tr, b_tr, c_tr = Xtr[:,0], Xtr[:,1], Xtr[:,2]
a_te, b_te, c_te = Xte[:,0], Xte[:,1], Xte[:,2]

In [7]:
a_tr.shape

(2000,)

In [9]:
# d: linear separierbar (einfaches lineares Decision Boundary)
# sign(0.8*a - 0.5*b + 0.3*c + 0.1)
def linear_target(a, b, c):
    z = 0.8*a - 0.5*b + 0.3*c + 0.1
    return (z > 0).astype(int)

# e: NICHT linear separierbar (XOR über die Vorzeichen von a und b)
def xor_target(a, b):
    sa, sb = (a > 0).astype(int), (b > 0).astype(int)
    return (sa ^ sb).astype(int)

d_tr = linear_target(a_tr, b_tr, c_tr)
e_tr = xor_target(a_tr, b_tr)

d_te = linear_target(a_te, b_te, c_te)
e_te = xor_target(a_te, b_te)

In [10]:
d_tr[:5]

array([1, 1, 1, 0, 0])

In [11]:
# Targets zu einer Matrix stapeln: Spalte 0 = d, Spalte 1 = e
T_tr = np.stack([d_tr, e_tr], axis=1)
T_te = np.stack([d_te, e_te], axis=1)

In [12]:
type(T_tr)

numpy.ndarray

In [15]:
T_tr[:5]

array([[1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 0]])

In [16]:
# Bias-Feature anhängen
def add_bias(X):
    return np.hstack([X, np.ones((X.shape[0], 1))])

Xtr_b = add_bias(Xtr)
Xte_b = add_bias(Xte)

In [17]:
Xtr_b[:5]

array([[ 0.30471708, -1.03998411,  0.7504512 ,  1.        ],
       [ 0.94056472, -1.95103519, -1.30217951,  1.        ],
       [ 0.1278404 , -0.31624259, -0.01680116,  1.        ],
       [-0.85304393,  0.87939797,  0.77779194,  1.        ],
       [ 0.0660307 ,  1.12724121,  0.46750934,  1.        ]])

In [20]:
n_features = Xtr_b.shape[1]   # 3 + Bias = 4
n_outputs = 2                 # d und e

# -----------------------------
# 2) Perzeptron-Training (klassische Lernregel)
# -----------------------------
# Initialisiere Gewichte (n_features x n_outputs)
W = rng.normal(scale=0.01, size=(n_features, n_outputs))

In [21]:
W

array([[-0.02533658, -0.01684464],
       [ 0.01554862, -0.01029192],
       [-0.00370835,  0.00655326],
       [ 0.0148229 , -0.00199587]])

In [ ]:
import numpy as np

# Reproduzierbarkeit
rng = np.random.default_rng(42)

# -----------------------------
# 1) Datengenerierung
# -----------------------------
# Eingaben: a, b, c ~ N(0,1)
N_train, N_test = 2000, 1000
Xtr = rng.normal(size=(N_train, 3))
Xte = rng.normal(size=(N_test, 3))

a_tr, b_tr, c_tr = Xtr[:,0], Xtr[:,1], Xtr[:,2]
a_te, b_te, c_te = Xte[:,0], Xte[:,1], Xte[:,2]

# d: linear separierbar (einfaches lineares Decision Boundary)
# sign(0.8*a - 0.5*b + 0.3*c + 0.1)
def linear_target(a, b, c):
    z = 0.8*a - 0.5*b + 0.3*c + 0.1
    return (z > 0).astype(int)

# e: NICHT linear separierbar (XOR über die Vorzeichen von a und b)
def xor_target(a, b):
    sa, sb = (a > 0).astype(int), (b > 0).astype(int)
    return (sa ^ sb).astype(int)

d_tr = linear_target(a_tr, b_tr, c_tr)
e_tr = xor_target(a_tr, b_tr)

d_te = linear_target(a_te, b_te, c_te)
e_te = xor_target(a_te, b_te)

# Targets zu einer Matrix stapeln: Spalte 0 = d, Spalte 1 = e
T_tr = np.stack([d_tr, e_tr], axis=1)
T_te = np.stack([d_te, e_te], axis=1)

# Bias-Feature anhängen
def add_bias(X):
    return np.hstack([X, np.ones((X.shape[0], 1))])

Xtr_b = add_bias(Xtr)
Xte_b = add_bias(Xte)

n_features = Xtr_b.shape[1]   # 3 + Bias = 4
n_outputs = 2                 # d und e

# -----------------------------
# 2) Perzeptron-Training (klassische Lernregel)
# -----------------------------
# Initialisiere Gewichte (n_features x n_outputs)
W = rng.normal(scale=0.01, size=(n_features, n_outputs))

def predict_logits(X, W):
    return X @ W

def predict_classes(X, W):
    # Schritt-Funktion (Heaviside): >0 -> 1, sonst 0
    return (predict_logits(X, W) > 0).astype(int)

def accuracy(Yhat, Y):
    return (Yhat == Y).mean(axis=0)  # Genauigkeit je Output-Spalte

lr = 1.0
epochs = 50

for epoch in range(epochs):
    # Online-Update (stochastisches Perzeptron)
    # Beispielreihenfolge mischen
    idx = rng.permutation(N_train)
    for i in idx:
        x = Xtr_b[i]                      # (4,)
        t = T_tr[i]                       # (2,)
        y = (x @ W > 0).astype(int)       # (2,)
        err = t - y                       # (2,)
        # Perzeptron-Regel: W <- W + lr * x[:,None] * err[None,:]
        W += lr * np.outer(x, err)

    # Monitoring
    Ytr_hat = predict_classes(Xtr_b, W)
    Yte_hat = predict_classes(Xte_b, W)
    tr_acc = accuracy(Ytr_hat, T_tr)
    te_acc = accuracy(Yte_hat, T_te)

    # Optional: frühes Stoppen, wenn beide Outputs perfekt sind (hier unrealistisch wegen XOR)
    # if np.all(tr_acc == 1.0):
    #     break

    if (epoch+1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:2d} | Train acc d={tr_acc[0]:.3f}, e={tr_acc[1]:.3f} | "
              f"Test acc d={te_acc[0]:.3f}, e={te_acc[1]:.3f}")

# -----------------------------
# 3) Finale Auswertung
# -----------------------------
Yte_hat = predict_classes(Xte_b, W)
te_acc_final = accuracy(Yte_hat, T_te)
print("\nFinale Test-Genauigkeit:")
print(f"  d (linear):      {te_acc_final[0]:.3f}")
print(f"  e (nicht-linear): {te_acc_final[1]:.3f}")

# Ein paar Beispiellabels zeigen
print("\nBeispielvorhersagen (erste 10):")
for i in range(10):
    x = Xte_b[i]
    pred = (x @ W > 0).astype(int)
    print(f"#{i:02d}  d_true={T_te[i,0]} d_pred={pred[0]} | e_true={T_te[i,1]} e_pred={pred[1]}")
